In [389]:
from urllib.request import urlretrieve
import zipfile
import pandas as pd
import numpy as np

users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    'data/u.user', sep='|', names=users_cols, encoding='latin-1')

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'data/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western",
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'data/u.item', sep='|', names=movies_cols, encoding='latin-1')

# Since the ids start at 1, we shift them to start at 0.
users["user_id"] = users["user_id"].apply(lambda x: str(x-1))
movies["movie_id"] = movies["movie_id"].apply(lambda x: str(x-1))
movies["year"] = movies['release_date'].apply(lambda x: str(x).split('-')[-1])
ratings["movie_id"] = ratings["movie_id"].apply(lambda x: str(x-1))
ratings["user_id"] = ratings["user_id"].apply(lambda x: str(x-1))
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))


genre_occurences = movies[genre_cols].sum().to_dict()

genres_encoded = {x: i for i, x in enumerate(genre_cols)}



def get_genres(movies, genres):
    def get_all_genres(gs):
        active = [str(genres_encoded[genre]) for genre, g in zip(genres, gs) if g==1]
        if len(active) == 0:
            return '0'
        return ','.join((active))
    movies['all_genres'] = [
        get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]

get_genres(movies, genre_cols) # 각 유저가 본 장르 얻기

rating_details_sample = ratings.merge(movies, on='movie_id').merge(users, on='user_id')

rating_details_sample['user_id']=rating_details_sample['user_id'].astype(int)
rating_details_sample['movie_id']=rating_details_sample['movie_id'].astype(int)
rating_details_sample=rating_details_sample.set_index(['user_id','unix_timestamp']).sort_index()
rating_details_sample =rating_details_sample.reset_index()

rating_details_sample['movie_type']=np.where(rating_details_sample['rating'] >= 3, 'like', 'dislike') # 3보다 크면 like
rating_details_sample['movie_name']=rating_details_sample['title'].str[:-6] # 년도 부분 자르기

user_ids = rating_details_sample["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

movie_ids = rating_details_sample["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}


title_ids = rating_details_sample["movie_name"].unique().tolist()
title2title_encoded = {x: i for i, x in enumerate(title_ids)}
title_encoded2title = {i: x for i, x in enumerate(title_ids)}

rating_details_sample["user"] = rating_details_sample["user_id"].map(user2user_encoded)
rating_details_sample["movie"] = rating_details_sample["movie_id"].map(movie2movie_encoded)
rating_details_sample["title_d"] = rating_details_sample["movie_name"].map(title2title_encoded)

sample_data=rating_details_sample[['user','occupation','sex']]
sample_data=sample_data.reset_index()

movie_list = rating_details_sample.groupby(['user','movie_type'])['movie'].apply(list).reset_index()
title_list = rating_details_sample.groupby(['user'])['title_d'].apply(list).reset_index()
genre_list = rating_details_sample.groupby(['user'])['all_genres'].unique().apply(list).reset_index()

# Get the unique set of genre for all the users
genre_list['all_genres']=genre_list['all_genres'].apply(lambda x: list(set(','.join(x))) ) # 중복제거
genre_list['all_genres']=genre_list['all_genres'].apply(lambda x:[ x for x in x if x.isdigit() ])

user_video_list = movie_list.pivot(index='user', columns='movie_type', values='movie').reset_index()
user_video_list.fillna(rating_details_sample["movie"].max()+1, inplace=True)

sample_data = sample_data.drop('index',axis=1)
sample_data = sample_data.drop_duplicates()

user_final_list = pd.merge(user_video_list,title_list, how= 'left')
user_title_list1 = pd.merge(user_final_list,genre_list, how='left')
user_title_list = pd.merge(user_title_list1,sample_data, how='left')

user_title_list['like'] =user_title_list['like'].apply(lambda x: x if type(x) is list else [x])
user_title_list['dislike'] =user_title_list['dislike'].apply(lambda x: x if type(x) is list else [x])
user_title_list['predict_labels'] = user_title_list['like'].apply(lambda x: (x[-1])) #label을 마지막 값으로..
# user_title_list['like']=user_title_list['like'].apply(lambda x: (x[:-1])) # 마지막 인덱스 레이블로 쓰고 제외
user_title_list['like']=user_title_list['like'].apply(lambda x: [rating_details_sample["movie"].max()+2] if x == [] else x)

# user_title_list['predict_labels'] = user_title_list['predict_labels'].apply(lambda x: 1 if x == 10 else x)

user_title_list_e=user_title_list[(user_title_list.user >= 1)&
                                  (user_title_list.user <= 500)]



In [390]:
user_title_list_e

,user,dislike,like,title_d,all_genres,occupation,sex,predict_labels
1,1,[10],"[4, 5]","[4, 5]","[6, 5, 2, 1, 8]",other,F,5
2,2,[10],"[0, 6]","[0, 6]","[6, 4, 8, 1, 5]",writer,M,6
3,3,[6],[10],[6],"[4, 8, 5]",technician,M,10
4,4,[7],[10],[7],"[8, 1, 5]",other,F,10
5,5,[10],[8],[8],[8],executive,M,8
6,6,[10],"[5, 6]","[5, 6]","[6, 4, 2, 8, 1, 5]",administrator,M,6
7,7,[10],"[2, 9, 4]","[2, 9, 4]","[4, 5, 3, 7, 1, 8]",administrator,M,4
8,8,[9],[10],[9],"[4, 5, 3]",student,M,10
9,9,[9],[10],[9],"[4, 5, 3]",lawyer,M,10


In [391]:
EMBEDDING_DIMS = 16
DENSE_UNITS = 64
DROPOUT_PCT = 0.0
ALPHA = 0.0
NUM_CLASSES=rating_details_sample["movie"].max() + 3
LEARNING_RATE = 0.003

In [392]:
import tensorflow as tf
class MaskedEmbeddingsAggregatorLayer(tf.keras.layers.Layer):
    def __init__(self, agg_mode='sum', **kwargs):
        super(MaskedEmbeddingsAggregatorLayer, self).__init__(**kwargs)

        if agg_mode not in ['sum', 'mean']:
            raise NotImplementedError('mode {} not implemented!'.format(agg_mode))
        self.agg_mode = agg_mode
    
    @tf.function
    def call(self, inputs, mask=None):
        masked_embeddings = tf.ragged.boolean_mask(inputs, mask)
        if self.agg_mode == 'sum':
            aggregated =  tf.reduce_sum(masked_embeddings, axis=1)
        elif self.agg_mode == 'mean':
            aggregated = tf.reduce_mean(masked_embeddings, axis=1)
        return aggregated
    
    def get_config(self):
        # this is used when loading a saved model that uses a custom layer
        return {'agg_mode': self.agg_mode}
    
class L2NormLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(L2NormLayer, self).__init__(**kwargs)
    
    @tf.function
    def call(self, inputs, mask=None):
        if mask is not None:
            inputs = tf.ragged.boolean_mask(inputs, mask).to_tensor()
        return tf.math.l2_normalize(inputs, axis=-1)

    def compute_mask(self, inputs, mask):
        return mask

In [393]:
#---inputs
import tensorflow as tf
import datetime
import os
input_title = tf.keras.Input(shape=(None, ), name='movie_name')
inp_video_liked = tf.keras.layers.Input(shape=(None,), name='like')
inp_video_disliked = tf.keras.layers.Input(shape=(None,), name='dislike')
input_genre = tf.keras.Input(shape=(None, ), name='genre')


#--- layers
features_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='features_embeddings')
labels_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='labels_embeddings')

avg_embeddings = MaskedEmbeddingsAggregatorLayer(agg_mode='mean', name='aggregate_embeddings')

dense_1 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_1')
dense_2 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_2')
dense_3 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_3')
l2_norm_1 = L2NormLayer(name='l2_norm_1')

dense_output = tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, name='dense_output')

#--- features
features_embeddings = features_embedding_layer(input_title)
l2_norm_features = l2_norm_1(features_embeddings)
avg_features = avg_embeddings(l2_norm_features)

labels_liked_embeddings = labels_embedding_layer(inp_video_liked)
l2_norm_liked = l2_norm_1(labels_liked_embeddings)
avg_liked = avg_embeddings(l2_norm_liked)

labels_disliked_embeddings = labels_embedding_layer(inp_video_disliked)
l2_norm_disliked = l2_norm_1(labels_disliked_embeddings)
avg_disliked = avg_embeddings(l2_norm_disliked)

labels_genre_embeddings = labels_embedding_layer(input_genre)
l2_norm_genre = l2_norm_1(labels_genre_embeddings)
avg_genre = avg_embeddings(l2_norm_genre)


# 임베딩 벡터들 연결
concat_inputs = tf.keras.layers.Concatenate(axis=1)([avg_features,
                                                     avg_liked,
                                                     avg_disliked,
                                                     avg_genre
                                                     ])
# Dense Layers
dense_1_features = dense_1(concat_inputs)
dense_1_relu = tf.keras.layers.ReLU(name='dense_1_relu')(dense_1_features)
dense_1_batch_norm = tf.keras.layers.BatchNormalization(name='dense_1_batch_norm')(dense_1_relu)

dense_2_features = dense_2(dense_1_relu)
dense_2_relu = tf.keras.layers.ReLU(name='dense_2_relu')(dense_2_features)
# dense_2_batch_norm = tf.keras.layers.BatchNormalization(name='dense_2_batch_norm')(dense_2_relu)

dense_3_features = dense_3(dense_2_relu)
dense_3_relu = tf.keras.layers.ReLU(name='dense_3_relu')(dense_3_features)
dense_3_batch_norm = tf.keras.layers.BatchNormalization(name='dense_3_batch_norm')(dense_3_relu)
outputs = dense_output(dense_3_batch_norm)

#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

#--- prep model
model = tf.keras.models.Model(
    inputs=[input_title, inp_video_liked, 
            inp_video_disliked,
            input_genre
            ],
    outputs=[outputs]
)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model.summary()

In [396]:
model.fit([tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['title_d']),
           tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['like']),
           tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['dislike']),
            tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['all_genres'])
           ],user_title_list_e['predict_labels'].values,
           steps_per_epoch=1, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 2/1000
1/1 [==============================] - 0s 912us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 3/1000
1/1 [==============================] - 0s 918us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 4/1000
1/1 [==============================] - 0s 813us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 5/1000
1/1 [==============================] - 0s 866us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 6/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 7/1000
1/1 [==============================] - 0s 814us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 8/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 9/1000
1/1 [==============================] - 0s 780us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 10/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 1

1/1 [==============================] - 0s 1ms/step - loss: 5.8320e-04 - accuracy: 1.0000
Epoch 81/1000
1/1 [==============================] - 0s 1ms/step - loss: 5.7833e-04 - accuracy: 1.0000
Epoch 82/1000
1/1 [==============================] - 0s 1ms/step - loss: 5.7362e-04 - accuracy: 1.0000
Epoch 83/1000
1/1 [==============================] - 0s 2ms/step - loss: 5.6901e-04 - accuracy: 1.0000
Epoch 84/1000
1/1 [==============================] - 0s 875us/step - loss: 5.6443e-04 - accuracy: 1.0000
Epoch 85/1000
1/1 [==============================] - 0s 767us/step - loss: 5.6006e-04 - accuracy: 1.0000
Epoch 86/1000
1/1 [==============================] - 0s 877us/step - loss: 5.5575e-04 - accuracy: 1.0000
Epoch 87/1000
1/1 [==============================] - 0s 1ms/step - loss: 5.5147e-04 - accuracy: 1.0000
Epoch 88/1000
1/1 [==============================] - 0s 1ms/step - loss: 5.4732e-04 - accuracy: 1.0000
Epoch 89/1000
1/1 [==============================] - 0s 2ms/step - loss: 5.4325e-

1/1 [==============================] - 0s 861us/step - loss: 3.5269e-04 - accuracy: 1.0000
Epoch 159/1000
1/1 [==============================] - 0s 805us/step - loss: 3.5070e-04 - accuracy: 1.0000
Epoch 160/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.4878e-04 - accuracy: 1.0000
Epoch 161/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.4685e-04 - accuracy: 1.0000
Epoch 162/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.4497e-04 - accuracy: 1.0000
Epoch 163/1000
1/1 [==============================] - 0s 997us/step - loss: 3.4310e-04 - accuracy: 1.0000
Epoch 164/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.4122e-04 - accuracy: 1.0000
Epoch 165/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.3938e-04 - accuracy: 1.0000
Epoch 166/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.3752e-04 - accuracy: 1.0000
Epoch 167/1000
1/1 [==============================] - 0s 1ms/step - loss:

1/1 [==============================] - 0s 1ms/step - loss: 2.3880e-04 - accuracy: 1.0000
Epoch 237/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.3771e-04 - accuracy: 1.0000
Epoch 238/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.3664e-04 - accuracy: 1.0000
Epoch 239/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.3557e-04 - accuracy: 1.0000
Epoch 240/1000
1/1 [==============================] - 0s 904us/step - loss: 2.3450e-04 - accuracy: 1.0000
Epoch 241/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.3349e-04 - accuracy: 1.0000
Epoch 242/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.3239e-04 - accuracy: 1.0000
Epoch 243/1000
1/1 [==============================] - 0s 927us/step - loss: 2.3141e-04 - accuracy: 1.0000
Epoch 244/1000
1/1 [==============================] - 0s 792us/step - loss: 2.3032e-04 - accuracy: 1.0000
Epoch 245/1000
1/1 [==============================] - 0s 936us/step - los

1/1 [==============================] - 0s 1ms/step - loss: 1.7228e-04 - accuracy: 1.0000
Epoch 315/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.7163e-04 - accuracy: 1.0000
Epoch 316/1000
1/1 [==============================] - 0s 904us/step - loss: 1.7100e-04 - accuracy: 1.0000
Epoch 317/1000
1/1 [==============================] - 0s 812us/step - loss: 1.7034e-04 - accuracy: 1.0000
Epoch 318/1000
1/1 [==============================] - 0s 2ms/step - loss: 1.6967e-04 - accuracy: 1.0000
Epoch 319/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.6904e-04 - accuracy: 1.0000
Epoch 320/1000
1/1 [==============================] - 0s 2ms/step - loss: 1.6835e-04 - accuracy: 1.0000
Epoch 321/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.6774e-04 - accuracy: 1.0000
Epoch 322/1000
1/1 [==============================] - 0s 866us/step - loss: 1.6705e-04 - accuracy: 1.0000
Epoch 323/1000
1/1 [==============================] - 0s 2ms/step - loss:

...

1/1 [==============================] - 0s 1ms/step - loss: 6.6516e-05 - accuracy: 1.0000
Epoch 627/1000
1/1 [==============================] - 0s 1ms/step - loss: 6.6318e-05 - accuracy: 1.0000
Epoch 628/1000
1/1 [==============================] - 0s 1ms/step - loss: 6.6199e-05 - accuracy: 1.0000
Epoch 629/1000
1/1 [==============================] - 0s 1ms/step - loss: 6.6026e-05 - accuracy: 1.0000
Epoch 630/1000
1/1 [==============================] - 0s 932us/step - loss: 6.5841e-05 - accuracy: 1.0000
Epoch 631/1000
1/1 [==============================] - 0s 1ms/step - loss: 6.5709e-05 - accuracy: 1.0000
Epoch 632/1000
1/1 [==============================] - 0s 1ms/step - loss: 6.5523e-05 - accuracy: 1.0000
Epoch 633/1000
1/1 [==============================] - 0s 1ms/step - loss: 6.5391e-05 - accuracy: 1.0000
Epoch 634/1000
1/1 [==============================] - 0s 2ms/step - loss: 6.5205e-05 - accuracy: 1.0000
Epoch 635/1000
1/1 [==============================] - 0s 784us/step - loss: 6

1/1 [==============================] - 0s 844us/step - loss: 5.5364e-05 - accuracy: 1.0000
Epoch 705/1000
1/1 [==============================] - 0s 1ms/step - loss: 5.5206e-05 - accuracy: 1.0000
Epoch 706/1000
1/1 [==============================] - 0s 1ms/step - loss: 5.5113e-05 - accuracy: 1.0000
Epoch 707/1000
1/1 [==============================] - 0s 2ms/step - loss: 5.4980e-05 - accuracy: 1.0000
Epoch 708/1000
1/1 [==============================] - 0s 868us/step - loss: 5.4848e-05 - accuracy: 1.0000
Epoch 709/1000
1/1 [==============================] - 0s 1ms/step - loss: 5.4795e-05 - accuracy: 1.0000
Epoch 710/1000
1/1 [==============================] - 0s 1ms/step - loss: 5.4636e-05 - accuracy: 1.0000
Epoch 711/1000
1/1 [==============================] - 0s 819us/step - loss: 5.4490e-05 - accuracy: 1.0000
Epoch 712/1000
1/1 [==============================] - 0s 886us/step - loss: 5.4411e-05 - accuracy: 1.0000
Epoch 713/1000
1/1 [==============================] - 0s 1ms/step - los

1/1 [==============================] - 0s 886us/step - loss: 4.6729e-05 - accuracy: 1.0000
Epoch 783/1000
1/1 [==============================] - 0s 829us/step - loss: 4.6610e-05 - accuracy: 1.0000
Epoch 784/1000
1/1 [==============================] - 0s 885us/step - loss: 4.6517e-05 - accuracy: 1.0000
Epoch 785/1000
1/1 [==============================] - 0s 1ms/step - loss: 4.6398e-05 - accuracy: 1.0000
Epoch 786/1000
1/1 [==============================] - 0s 781us/step - loss: 4.6305e-05 - accuracy: 1.0000
Epoch 787/1000
1/1 [==============================] - 0s 853us/step - loss: 4.6199e-05 - accuracy: 1.0000
Epoch 788/1000
1/1 [==============================] - 0s 976us/step - loss: 4.6133e-05 - accuracy: 1.0000
Epoch 789/1000
1/1 [==============================] - 0s 1ms/step - loss: 4.6014e-05 - accuracy: 1.0000
Epoch 790/1000
1/1 [==============================] - 0s 887us/step - loss: 4.5961e-05 - accuracy: 1.0000
Epoch 791/1000
1/1 [==============================] - 0s 1ms/step

1/1 [==============================] - 0s 1ms/step - loss: 3.9709e-05 - accuracy: 1.0000
Epoch 861/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.9603e-05 - accuracy: 1.0000
Epoch 862/1000
1/1 [==============================] - 0s 979us/step - loss: 3.9510e-05 - accuracy: 1.0000
Epoch 863/1000
1/1 [==============================] - 0s 854us/step - loss: 3.9457e-05 - accuracy: 1.0000
Epoch 864/1000
1/1 [==============================] - 0s 817us/step - loss: 3.9338e-05 - accuracy: 1.0000
Epoch 865/1000
1/1 [==============================] - 0s 2ms/step - loss: 3.9259e-05 - accuracy: 1.0000
Epoch 866/1000
1/1 [==============================] - 0s 904us/step - loss: 3.9193e-05 - accuracy: 1.0000
Epoch 867/1000
1/1 [==============================] - 0s 932us/step - loss: 3.9140e-05 - accuracy: 1.0000
Epoch 868/1000
1/1 [==============================] - 0s 881us/step - loss: 3.9100e-05 - accuracy: 1.0000
Epoch 869/1000
1/1 [==============================] - 0s 1ms/step -

1/1 [==============================] - 0s 897us/step - loss: 3.4199e-05 - accuracy: 1.0000
Epoch 938/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.4133e-05 - accuracy: 1.0000
Epoch 939/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.4014e-05 - accuracy: 1.0000
Epoch 940/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.3974e-05 - accuracy: 1.0000
Epoch 941/1000
1/1 [==============================] - 0s 801us/step - loss: 3.3934e-05 - accuracy: 1.0000
Epoch 942/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.3855e-05 - accuracy: 1.0000
Epoch 943/1000
1/1 [==============================] - 0s 880us/step - loss: 3.3828e-05 - accuracy: 1.0000
Epoch 944/1000
1/1 [==============================] - 0s 849us/step - loss: 3.3789e-05 - accuracy: 1.0000
Epoch 945/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.3722e-05 - accuracy: 1.0000
Epoch 946/1000
1/1 [==============================] - 0s 829us/step - l

In [384]:
results = model.evaluate([tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['title_d']),
           tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['like']),
           tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['dislike']),
            tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['all_genres'])
           ],user_title_list_e['predict_labels'].values, verbose=2)

1/1 - 0s - loss: nan - accuracy: 0.0000e+00
